# TensorFlow Dataset API

**目的**
* tf.data を使ってデータを TF モデルを読み込む方法を学ぶ
* インメモリに全てのデータをロードする方法と、ディスクからバッチでロードする方法の違いを学ぶ
* tf.data を使ってcsv ファイルをディスクからバッチで読み込む方法を学習する

このノートブックでは、`tf.data.Dataset` を使用したデータ入力パイプラインの作成方法を学びます<br>
データをバッチで読み込めるようにし、 後にそのデータに対して**確率的勾配降下法**を利用できる準備をします。


In [ ]:
import json
import math
import os
from pprint import pprint

import numpy as np
import tensorflow as tf
print(tf.version.VERSION)

## データをメモリからロードする

### データセットを作成する
では、前のラボで生成したデータをまた作成しましょう

In [ ]:
N_POINTS = 10
# The .constant() method will creates a constant tensor from a tensor-like object.
X = tf.constant(range(N_POINTS), dtype=tf.float32)
Y = 2 * X + 10

tf.Data.Datasetを出力する関数を作成します。この関数は、以下を引数として取得します。

- $y= 2x + 10$ という線形関数で合成された $X$ と $Y$ のベクトル
- データセットを使用してトレーニングを行う回数 (epochs)
- バッチサイズ (batch_size)

**注意**: データセットから取得する最後のバッチのデータ数は、指定したバッチサイズと異なる可能性があります。

すべてのバッチを同じサイズにしたい場合、以下のように最後のバッチを破棄する必要があります。

```python
dataset = dataset.batch(batch_size, drop_remainder=True)
```

In [ ]:
def create_dataset(X, Y, epochs, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.repeat(epochs).batch(batch_size, drop_remainder=True)
    return dataset


では、この関数を使用して、私たちのデータセットを3つのデータ数のバッチにしながら2回イテレーションを回しましょう。

In [ ]:
BATCH_SIZE = 3
EPOCH = 2

dataset = create_dataset(X, Y, epochs=EPOCH, batch_size=BATCH_SIZE)

for i, (x, y) in enumerate(dataset):
    print("x:", x.numpy(), "y:", y.numpy())
    assert len(x) == BATCH_SIZE
    assert len(y) == BATCH_SIZE
assert  EPOCH

### 損失関数と勾配
損失や勾配を計算する関数は前のセクションと同じです。

In [ ]:
def loss_mse(X, Y, w0, w1):
    Y_hat = w0 * X + w1
    errors = (Y_hat - Y)**2
    return tf.reduce_mean(errors)

def compute_gradients(X, Y, w0, w1):
    with tf.GradientTape() as tape:
        loss = loss_mse(X, Y, w0, w1)
    return tape.gradient(loss, [w0, w1])

### トレーニングループ
ここでの大きな違いは、トレーニングループの中で、`create_dataset`関数が出力する `tf.data.Dataset` を直接イテレーションすることです。

In [ ]:
EPOCHS = 1000
BATCH_SIZE = 2
LEARNING_RATE = .02

MSG = "STEP {step} - loss: {loss}, w0: {w0}, w1: {w1}\n"

w0 = tf.Variable(0.0)
w1 = tf.Variable(0.0)

dataset = create_dataset(X, Y, epochs=EPOCHS, batch_size=BATCH_SIZE)

for step, (X_batch, Y_batch) in enumerate(dataset):

    dw0, dw1 = compute_gradients(X_batch, Y_batch, w0, w1)
    w0.assign_sub(dw0 * LEARNING_RATE)
    w1.assign_sub(dw1 * LEARNING_RATE)

    if step % 100 == 0:
        loss = loss_mse(X_batch, Y_batch, w0, w1)
        print(MSG.format(step=step, loss=loss, w0=w0.numpy(), w1=w1.numpy()))
        

## データをディスクからロードする

では、先ほど作成したデータセットをロードしていきましょう。

まずは、いくつかのデータを確認してみましょう。
カラム名とその順序に注意してください。


In [ ]:
!head ../data/taxi-*

### tf.data を使用して CSV ファイルを読み込む

`tf.data` API は関数を使用して簡単に CSV ファイルを読み込むことができます。

[tf.data.experimental.make_csv_dataset](https://www.tensorflow.org/api_docs/python/tf/data/experimental/make_csv_dataset)


TFRecord 形式のデータがある場合には、以下を使用します。

[tf.data.experimental.make_batched_features_dataset](https://www.tensorflow.org/api_docs/python/tf/data/experimental/make_batched_features_dataset)

最初のステップは、

- 特徴量列の名前を `CSV_COLUMNS` リストに格納する
- デフォルトの値を `DEFAULTS` リストに格納する

In [ ]:
CSV_COLUMNS = ['fare_amount',
            'dayofweek',
            'hourofday',
            'pickup_longitude',
            'pickup_latitude',
            'dropoff_longitude',
            'dropoff_latitude',
           ]
LABEL_COLUMN = 'fare_amount'
DEFAULTS = [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]

では、`make_csv_dataset` を独自の関数でラッピングし、glob などのファイルパターンのパスからデータを取得できるようにしましょう。

In [ ]:
def create_dataset(pattern):
    return tf.data.experimental.make_csv_dataset(
        pattern, 1, CSV_COLUMNS, DEFAULTS)

tempds = create_dataset('../data/taxi-train*')
print(tempds)

このデータがプリフェッチされたデータセットであることに注意してください。このデータは、各要素をキーに特徴量名をとり、値に`(1,)`の形のTensor をとる`OrderedDict` 形式です。

では、このデータセットの最初の二つの要素を `dataset.take(2)` を用いて取得し、numpy arrayを値に持つ通常の Python 辞書型に変換してみましょう。

In [ ]:
for data in tempds.take(2):
    pprint({k: v.numpy() for k, v in data.items()})
    print("\n")

### 特徴量を変換する

ここで必要なのは、特徴量とラベルですので、この辞書に二つのことをする必要があります。

1. 不必要な列を除外する。（ここでは存在しませんが、何をするべきかは定義しておきます）
1. ラベルを特徴量から区別して保持する

まず、一つのレコード（`tf.data.Dataset` に含まれる `OrderedDict` の形式）を入力に取り、以下の二つの要素を含むタプルを出力する関数を定義しましょう。

- `OrderedDict` からラベルを除外したもの
- ラベル (`fare_amount`)

In [ ]:
UNWANTED_COLS = []


def features_and_labels(row_data):
    label = row_data.pop(LABEL_COLUMN)
    features = row_data
    
    for unwanted_col in UNWANTED_COLS:
        features.pop(unwanted_col)

    return features, label

`tempds` データセット内の二つのデータポイントに対して、 試しに`feature_and_labels` 関数を実行してテストしてみましょう。

In [ ]:
for row_data in tempds.take(2):
    features, label = features_and_labels(row_data)
    pprint(features)
    print(label, "\n")
    
    assert label.shape == [1]

### ミニバッチ処理

`create_dataset` 関数をリファクタリングし、`batch_size` を引数にとってデータをバッチに分ける機能を追加しましょう。
また、実装した`features_and_labels`関数を使用して、特徴量とラベルのタプルを生成しましょう。


In [ ]:
def create_dataset(pattern, batch_size):
    dataset = tf.data.experimental.make_csv_dataset(
        pattern, batch_size, CSV_COLUMNS, DEFAULTS)
    return dataset.map(features_and_labels)

バッチが正しいサイズになっているかどうかテストしましょう。

In [ ]:
BATCH_SIZE = 2

tempds = create_dataset('../data/taxi-train*', batch_size=BATCH_SIZE)

for X_batch, Y_batch in tempds.take(2):
    pprint({k: v.numpy() for k, v in X_batch.items()})
    print(Y_batch.numpy(), "\n")
    assert len(Y_batch) == BATCH_SIZE

### シャッフル

ディープラーニングのモデルを複数のワーカーでミニバッチ学習をする際、データをシャッフルするのが有効です。

`create_dataset` 関数をリファクタリングして、データのシャッフルが行えるようにしましょう。

ここで、関数の内部で場合分けをすることができるように、新たに `mode` という引数を追加しましょう。<br>
たとえば、トレーニング(`mode == tf.estimator.ModeKeys.TRAIN`)の際にはデータをシャッフルし、検証時(`mode == tf.estimator.ModeKeys.EVAL`)にはシャッフルは行わない場合には、以下のように書くことができます

In [ ]:
def create_dataset(pattern, batch_size=1, mode=tf.estimator.ModeKeys.EVAL):
    dataset = tf.data.experimental.make_csv_dataset(
        pattern, batch_size, CSV_COLUMNS, DEFAULTS)

    dataset = dataset.map(features_and_labels).cache()

    if mode == tf.estimator.ModeKeys.TRAIN:
        dataset = dataset.shuffle(1000).repeat()

    # take advantage of multi-threading; 1=AUTOTUNE
    dataset = dataset.prefetch(1)
    return dataset

この二つのモードがうまく動いているか確認しましょう。

In [ ]:
tempds = create_dataset('../data/taxi-train*', 2, tf.estimator.ModeKeys.TRAIN)
print(list(tempds.take(1)))

In [ ]:
tempds = create_dataset('../data/taxi-valid*', 2, tf.estimator.ModeKeys.EVAL)
print(list(tempds.take(1)))

これで、データのパイプラインの作り方を学習することができました。<br>
次のノートブックでは、この入力パイプラインを利用してモデルを構築します。

Copyright 2021 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.